### 2.4 Create PyTorch Datasets and DataLoaders

In [1]:
# Import required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import h5py
import os
import datetime
from pathlib import Path
from tqdm.notebook import tqdm

# PyTorch libraries
import torch
from torch.utils.data import DataLoader
import wandb

from utils.data_persistence import load_scalers
from utils.training_utils import plot_training_history, plot_predictions
from utils.wandb_utils import setup_wandb
from utils.training_utils import train_model, evaluate_model
from utils.wandb_utils import setup_wandb

# Set random seeds for reproducibility
np.random.seed(42)
torch.manual_seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(42)

# Set device to GPU if available, otherwise use CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device} device")


# List of features to use
AVAILABLE_FEATURES = [
    'ghi',                     # Target variable
    'air_temperature',         # Weather features
    'wind_speed',
    'relative_humidity',
    'dew_point',
    'surface_pressure',
    'total_precipitable_water',
    'cloud_type',              # Cloud features
    'cloud_fill_flag',
    'cld_opd_dcomp',
    'cld_press_acha',
    'cld_reff_dcomp',
    'clearsky_ghi',            # Clear sky estimates
    'clearsky_dni',
    'clearsky_dhi',
    'solar_zenith_angle',      # Solar geometry
    'surface_albedo',          # Surface properties
    'ozone',                   # Atmospheric properties
    'aod',
    'ssa',
    'asymmetry',
    'alpha'
]

# Choose features to use in modeling
SELECTED_FEATURES = [
    'air_temperature',
    'wind_speed',
    'relative_humidity',
    'cloud_type',
    'solar_zenith_angle',
    'clearsky_ghi',
    'total_precipitable_water',
    'surface_albedo'
]

# Target variable
TARGET_VARIABLE = 'ghi'


Using cpu device


In [2]:
from utils.data_persistence import load_normalized_data

train_preprocessed_data_path = "data/processed/train_normalized.h5"
val_preprocessed_data_path = "data/processed/val_normalized.h5"
test_preprocessed_data_path = "data/processed/test_normalized.h5"

# Load sequences
train_data, metadata = load_normalized_data(train_preprocessed_data_path)

scaler_path = "data/processed/model_scalers.pkl"
scalers = load_scalers(scaler_path)

# Print metadata
print(f"Train set | Metadata: {metadata}")
# Print created time
print(f"Train set | Created time: {metadata['created_time'] if 'created_time' in metadata else 'No created time'}")
# Print raw files
print(f"Train set | Raw files: {metadata['raw_files'] if 'raw_files' in metadata else 'No raw files'}")

# Print data structure and shape
print(f"Train set | Data structure:")
for key, value in train_data.items():
    print(f"  {key} shape: {value.shape}")


Loaded normalized data from data/processed/train_normalized.h5
Loaded 12 scalers from data/processed/model_scalers.pkl
Train set | Metadata: {'created_time': '2025-04-25 05:36:18'}
Train set | Created time: 2025-04-25 05:36:18
Train set | Raw files: No raw files
Train set | Data structure:
  air_temperature shape: (8760, 105)
  clearsky_ghi shape: (8760, 105)
  cloud_type shape: (8760, 105)
  coordinates shape: (105, 2)
  elevation shape: (105,)
  ghi shape: (8760, 105)
  nighttime_mask shape: (8760, 105)
  relative_humidity shape: (8760, 105)
  solar_zenith_angle shape: (8760, 105)
  surface_albedo shape: (8760, 105)
  time_features shape: (8760, 8)
  total_precipitable_water shape: (8760, 105)
  wind_speed shape: (8760, 105)


In [3]:
from utils.timeseriesdataset import TimeSeriesDataset

LOOKBACK = 24

# Create datasets
train_dataset = TimeSeriesDataset(train_preprocessed_data_path, lookback=LOOKBACK)
val_dataset = TimeSeriesDataset(val_preprocessed_data_path, lookback=LOOKBACK)
test_dataset = TimeSeriesDataset(test_preprocessed_data_path, lookback=LOOKBACK)

# Create data loaders
batch_size = 64
num_workers = 4
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers)

# Check sample batch
sample_batch = next(iter(train_loader))
for key, value in sample_batch.items():
    if isinstance(value, torch.Tensor):
        print(f"{key} shape: {value.shape}")


Loaded normalized data from data/processed/train_normalized.h5
Loaded normalized data file (1/1): data/processed/train_normalized.h5
Loaded data with 13 features
Temporal features: ['time_features']
Static features: ['coordinates', 'elevation']
Time series features: ['air_temperature', 'clearsky_ghi', 'cloud_type', 'ghi', 'nighttime_mask', 'relative_humidity', 'solar_zenith_angle', 'surface_albedo', 'total_precipitable_water', 'wind_speed']
Dataset dimensions: 8760 timesteps, 105 locations
Dataset contains 917280 possible samples
Loaded normalized data from data/processed/val_normalized.h5
Loaded normalized data file (1/1): data/processed/val_normalized.h5
Loaded data with 13 features
Temporal features: ['time_features']
Static features: ['coordinates', 'elevation']
Time series features: ['air_temperature', 'clearsky_ghi', 'cloud_type', 'ghi', 'nighttime_mask', 'relative_humidity', 'solar_zenith_angle', 'surface_albedo', 'total_precipitable_water', 'wind_speed']
Dataset dimensions: 876

## 3. Model Implementations

### 3.1 LSTM Model

In [4]:
# Get a batch to determine input dimensions
batch = next(iter(train_loader))

# Method 1: Extract dimensions from a batch (more reliable)
temporal_features = batch['temporal_features']
static_features = batch['static_features']

# Check if we have 3D temporal features (batch, seq_len, features)
if len(temporal_features.shape) == 3:
    temporal_dim = temporal_features.shape[2]
else:
    # Handle 2D temporal features (batch, features)
    temporal_dim = temporal_features.shape[1]

static_dim = static_features.shape[1]

print(f"  Input dimensions determined from batch:")
print(f"  - Temporal dimension: {temporal_dim}")
print(f"  - Static dimension: {static_dim}")


  Input dimensions determined from batch:
  - Temporal dimension: 8
  - Static dimension: 2


In [5]:
from models.lstm import LSTMModel

# Create LSTM model
lstm_model = LSTMModel(
    input_dim=temporal_dim,
    static_dim=static_dim,
    hidden_dim=128,
    num_layers=2,
    dropout=0.3
).to(device)

print(lstm_model)


LSTMModel(
  (lstm): LSTM(8, 128, num_layers=2, batch_first=True, dropout=0.3)
  (bn_lstm): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (static_proj): Sequential(
    (0): Linear(in_features=2, out_features=32, bias=True)
    (1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout(p=0.3, inplace=False)
  )
  (fc): Sequential(
    (0): Linear(in_features=160, out_features=64, bias=True)
    (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout(p=0.3, inplace=False)
    (4): Linear(in_features=64, out_features=32, bias=True)
    (5): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): Dropout(p=0.3, inplace=False)
    (8): Linear(in_features=32, out_features=1, bias=True)
  )
)


### 3.2 CNN-LSTM Model

In [6]:
from models.cnn_lstm import CNNLSTMModel

# Create CNN-LSTM model
cnn_lstm_model = CNNLSTMModel(
    input_dim=temporal_dim,
    static_dim=static_dim,
    hidden_dim=128,
    num_filters=64,
    kernel_size=3,
    num_layers=2,
    dropout=0.3
).to(device)

print(cnn_lstm_model)


CNNLSTMModel(
  (cnn): Sequential(
    (0): Conv1d(8, 64, kernel_size=(3,), stride=(1,), padding=(1,))
    (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv1d(64, 128, kernel_size=(3,), stride=(1,), padding=(1,))
    (5): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (lstm): LSTM(128, 128, num_layers=2, batch_first=True, dropout=0.3)
  (bn_lstm): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (static_proj): Sequential(
    (0): Linear(in_features=2, out_features=32, bias=True)
    (1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout(p=0.3, inplace=False)
  )
  (fc): Sequential(
    (0): Linear(in_featu

### 3.3 Multi-Layer Perceptron (MLP) Model

In [7]:
from models.mlp import MLPModel

# Create MLP model
mlp_model = MLPModel(
    input_dim=temporal_dim,
    static_dim=static_dim,
    hidden_dims=[256, 512, 256, 128],
    dropout=0.3,
    lookback=LOOKBACK
).to(device)

print(mlp_model)


MLPModel(
  (mlp): Sequential(
    (0): Linear(in_features=192, out_features=256, bias=True)
    (1): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout(p=0.3, inplace=False)
    (4): Linear(in_features=256, out_features=512, bias=True)
    (5): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): Dropout(p=0.3, inplace=False)
    (8): Linear(in_features=512, out_features=256, bias=True)
    (9): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU()
    (11): Dropout(p=0.3, inplace=False)
    (12): Linear(in_features=256, out_features=128, bias=True)
    (13): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (14): ReLU()
    (15): Dropout(p=0.3, inplace=False)
  )
  (static_proj): Sequential(
    (0): Linear(in_features=2, out_features=32, bias=True)
    (1): BatchNorm1d(32, eps=1e-05, momentu

## 4. Model Training and Evaluation

In [8]:
from utils.training_utils import train_model, evaluate_model
from utils.wandb_utils import setup_wandb

# Default settings
USE_WANDB = True
WANDB_USERNAME = "tin-hoang"
WANDB_PROJECT = "EEEM073-Solar-Radiation"
setup_wandb(WANDB_USERNAME, WANDB_PROJECT)


Weights & Biases tracking enabled with username 'tin-hoang' and project 'EEEM073-Solar-Radiation'


True

## 5. Train and Evaluate Models

### 5.1 Train and Evaluate LSTM Model

In [ ]:
print("Training LSTM model...")
lstm_history = train_model(
    lstm_model,
    train_loader,
    val_loader,
    model_name="LSTM",
    epochs=30,
    patience=5,
    lr=0.001
)
plot_training_history(lstm_history, model_name="LSTM")

print("Evaluating LSTM model on validation set...")
lstm_val_metrics = evaluate_model(
    lstm_model,
    val_loader,
    scalers[f'{TARGET_VARIABLE}_scaler'],
    model_name="LSTM - Validation"
)
plot_predictions(lstm_val_metrics, model_name='LSTM - Validation')

print("\nEvaluating LSTM model on test set...")
lstm_test_metrics = evaluate_model(
    lstm_model,
    test_loader,
    scalers[f'{TARGET_VARIABLE}_scaler'],
    model_name="LSTM - Test"
)
plot_predictions(lstm_test_metrics, model_name='LSTM - Test')

torch.save(lstm_model.state_dict(), 'lstm_ghi_forecasting_model_v2.pt')


Training LSTM model...


wandb: Currently logged in as: tin-hoang to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Training LSTM:   0%|                                                      | 0/30 [00:00<?, ?it/s]

### 5.2 Train and Evaluate CNN-LSTM Model

In [ ]:
print("Training CNN-LSTM model...")
cnn_lstm_history = train_model(
    cnn_lstm_model,
    train_loader,
    val_loader,
    model_name="CNN-LSTM",
    epochs=30,
    patience=5,
    lr=0.001
)
plot_training_history(cnn_lstm_history, model_name="CNN-LSTM")

print("Evaluating CNN-LSTM model on validation set...")
cnn_lstm_val_metrics = evaluate_model(
    cnn_lstm_model,
    val_loader,
    scalers[f'{TARGET_VARIABLE}_scaler'],
    model_name="CNN-LSTM - Validation"
)
plot_predictions(cnn_lstm_val_metrics, model_name='CNN-LSTM - Validation')

print("\nEvaluating CNN-LSTM model on test set...")
cnn_lstm_test_metrics = evaluate_model(
    cnn_lstm_model,
    test_loader,
    scalers[f'{TARGET_VARIABLE}_scaler'],
    model_name="CNN-LSTM - Test"
)
plot_predictions(cnn_lstm_test_metrics, model_name='CNN-LSTM - Test')

torch.save(cnn_lstm_model.state_dict(), 'cnn_lstm_ghi_forecasting_model_v2.pt')


### 5.3 Train and Evaluate MLP Model

In [ ]:
print("Training MLP model...")
mlp_history = train_model(
    mlp_model,
    train_loader,
    val_loader,
    model_name="MLP",
    epochs=30,
    patience=5,
    lr=0.001
)
plot_training_history(mlp_history, model_name="MLP")

print("Evaluating MLP model on validation set...")
mlp_val_metrics = evaluate_model(
    mlp_model,
    val_loader,
    scalers[f'{TARGET_VARIABLE}_scaler'],
    model_name="MLP - Validation"
)
plot_predictions(mlp_val_metrics, model_name='MLP - Validation')

print("\nEvaluating MLP model on test set...")
mlp_test_metrics = evaluate_model(
    mlp_model,
    test_loader,
    scalers[f'{TARGET_VARIABLE}_scaler'],
    model_name="MLP - Test"
)
plot_predictions(mlp_test_metrics, model_name='MLP - Test')

torch.save(mlp_model.state_dict(), 'mlp_ghi_forecasting_model_v2.pt')


## 6. Model Comparison

In [ ]:
def compare_models(metrics_list, model_names, dataset_name=""):
    """
    Compare performance metrics across models

    Args:
        metrics_list: List of metrics dictionaries
        model_names: List of model names
        dataset_name: Name of the dataset (train/val/test)
    """
    metrics = ['rmse', 'mae', 'r2', 'day_rmse', 'day_mae', 'day_r2', 'night_rmse', 'night_mae']
    metric_labels = ['RMSE', 'MAE', 'R²', 'Day RMSE', 'Day MAE', 'Day R²', 'Night RMSE', 'Night MAE']

    comparison = pd.DataFrame(index=metric_labels, columns=model_names)

    for i, model_metrics in enumerate(metrics_list):
        for j, metric in enumerate(metrics):
            if metric in model_metrics:
                comparison.iloc[j, i] = model_metrics[metric]
            else:
                comparison.iloc[j, i] = np.nan

    print(f"\nModel Comparison - {dataset_name} Set:")
    print(comparison)

    # Create comparison visualization
    fig_comparison = create_comparison_plots(metrics_list, model_names, dataset_name)
    plt.show()

    # Log comparison to wandb
    if USE_WANDB:
        # Create a comparison table
        comparison_table = wandb.Table(
            columns=["Metric"] + model_names,
            data=[[metric_labels[i]] + [comparison.iloc[i, j] for j in range(len(model_names))]
                  for i in range(len(metric_labels))]
        )

        # Log to wandb
        wandb.init(
            project=WANDB_PROJECT,
            entity=WANDB_USERNAME,
            name=f"Model-Comparison-{dataset_name}",
            config={"dataset": dataset_name}
        )

        wandb.log({
            f"comparison_table_{dataset_name}": comparison_table,
            f"comparison_plot_{dataset_name}": wandb.Image(fig_comparison)
        })

        # Create individual metric plots for clearer visualization
        for j, metric in enumerate(metrics[:6]):  # Skip night metrics which might be NaN
            plt.figure(figsize=(10, 6))
            values = [metrics_dict[metric] if not np.isnan(metrics_dict[metric]) else 0 for metrics_dict in metrics_list]
            bars = plt.bar(model_names, values)

            # Add values on top of bars
            for bar, value in zip(bars, values):
                plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
                        f"{value:.3f}" if metric.startswith('r2') else f"{value:.2f}",
                        ha='center', va='bottom')

            plt.title(f'{metric_labels[j]} Comparison - {dataset_name} Set')
            plt.ylabel(metric_labels[j])
            plt.grid(axis='y')
            plt.tight_layout()

            # Log to wandb
            wandb.log({f"{metric}_{dataset_name}_comparison": wandb.Image(plt)})
            plt.close()

        wandb.finish()

    return comparison

def create_comparison_plots(metrics_list, model_names, dataset_name=""):
    """
    Create comparison visualizations for multiple models

    Args:
        metrics_list: List of metrics dictionaries
        model_names: List of model names
        dataset_name: Name of the dataset (train/val/test)

    Returns:
        fig: Matplotlib figure
    """
    # Create grouped bar charts for key metrics
    # RMSE and MAE for overall, day, night
    fig, axes = plt.subplots(1, 3, figsize=(18, 6))

    # Overall metrics
    x = np.arange(len(model_names))
    width = 0.35

    # Plot RMSE
    rmse_values = [metrics_dict['rmse'] for metrics_dict in metrics_list]
    bars1 = axes[0].bar(x - width/2, rmse_values, width, label='RMSE')
    # Plot MAE
    mae_values = [metrics_dict['mae'] for metrics_dict in metrics_list]
    bars2 = axes[0].bar(x + width/2, mae_values, width, label='MAE')

    axes[0].set_title('Overall Error Metrics')
    axes[0].set_xticks(x)
    axes[0].set_xticklabels(model_names)
    axes[0].legend()
    axes[0].grid(axis='y')

    # Add data labels
    for i, v in enumerate(rmse_values):
        axes[0].text(i - width/2, v + 0.1, f"{v:.1f}", ha='center')
    for i, v in enumerate(mae_values):
        axes[0].text(i + width/2, v + 0.1, f"{v:.1f}", ha='center')

    # Daytime metrics
    day_rmse_values = [metrics_dict['day_rmse'] for metrics_dict in metrics_list]
    bars3 = axes[1].bar(x - width/2, day_rmse_values, width, label='Day RMSE')
    day_mae_values = [metrics_dict['day_mae'] for metrics_dict in metrics_list]
    bars4 = axes[1].bar(x + width/2, day_mae_values, width, label='Day MAE')

    axes[1].set_title('Daytime Error Metrics')
    axes[1].set_xticks(x)
    axes[1].set_xticklabels(model_names)
    axes[1].legend()
    axes[1].grid(axis='y')

    # Add data labels
    for i, v in enumerate(day_rmse_values):
        axes[1].text(i - width/2, v + 0.1, f"{v:.1f}", ha='center')
    for i, v in enumerate(day_mae_values):
        axes[1].text(i + width/2, v + 0.1, f"{v:.1f}", ha='center')

    # Nighttime metrics
    night_rmse_values = [metrics_dict['night_rmse'] for metrics_dict in metrics_list]
    bars5 = axes[2].bar(x - width/2, night_rmse_values, width, label='Night RMSE')
    night_mae_values = [metrics_dict['night_mae'] for metrics_dict in metrics_list]
    bars6 = axes[2].bar(x + width/2, night_mae_values, width, label='Night MAE')

    axes[2].set_title('Nighttime Error Metrics')
    axes[2].set_xticks(x)
    axes[2].set_xticklabels(model_names)
    axes[2].legend()
    axes[2].grid(axis='y')

    # Add data labels
    for i, v in enumerate(night_rmse_values):
        if not np.isnan(v):
            axes[2].text(i - width/2, v + 0.1, f"{v:.1f}", ha='center')
    for i, v in enumerate(night_mae_values):
        if not np.isnan(v):
            axes[2].text(i + width/2, v + 0.1, f"{v:.1f}", ha='center')

    plt.tight_layout()
    plt.suptitle(f'Model Comparison - {dataset_name} Set', fontsize=16)
    plt.subplots_adjust(top=0.9)

    return fig

# Compare model performance on validation set
print("Validation Set Comparison:")
compare_models(
    [lstm_val_metrics, cnn_lstm_val_metrics, mlp_val_metrics],
    ['LSTM', 'CNN-LSTM', 'MLP', 'PINN-MLP'],
    dataset_name="Validation"
)

# Compare model performance on test set
print("\nTest Set Comparison:")
compare_models(
    [lstm_test_metrics, cnn_lstm_test_metrics, mlp_test_metrics],
    ['LSTM', 'CNN-LSTM', 'MLP', 'PINN-MLP'],
    dataset_name="Test"
)


### 6.1 Time Series Predictions

Visualize predictions over time, including the PINN-MLP model.

In [ ]:
def plot_predictions_over_time(models, model_names, data_loader, target_scaler, num_samples=200, start_idx=0):
    """
    Plot time series predictions for multiple models

    Args:
        models: List of PyTorch models
        model_names: List of model names
        data_loader: Data loader
        target_scaler: Scaler for the target variable
        num_samples: Number of consecutive time steps to plot
        start_idx: Starting index in the dataset
    """
    # Collect data samples
    all_batches = []
    for batch in data_loader:
        all_batches.append(batch)
        if len(all_batches) * batch['target'].shape[0] > start_idx + num_samples:
            break

    # Combine batches into a single dataset
    all_temporal = []
    all_static = []
    all_targets = []

    for batch in all_batches:
        all_temporal.append(batch['temporal_features'])
        all_static.append(batch['static_features'])
        all_targets.append(batch['target'])

    all_temporal = torch.cat(all_temporal, dim=0)
    all_static = torch.cat(all_static, dim=0)
    all_targets = torch.cat(all_targets, dim=0)

    # Get the subset for visualization
    temporal = all_temporal[start_idx:start_idx+num_samples].to(device)
    static = all_static[start_idx:start_idx+num_samples].to(device)
    targets = all_targets[start_idx:start_idx+num_samples].cpu().numpy()

    # Generate predictions
    predictions = []
    for model in models:
        model.eval()
        with torch.no_grad():
            outputs = model(temporal, static).cpu().numpy()
            predictions.append(outputs)

    # Inverse transform to original scale
    y_true_orig = target_scaler.inverse_transform(targets)
    y_pred_orig_list = [target_scaler.inverse_transform(pred) for pred in predictions]

    # Create visualization
    fig = plt.figure(figsize=(15, 8))

    # Plot predictions
    plt.plot(y_true_orig, 'k-', label='Actual GHI', linewidth=2)

    colors = ['b-', 'r-', 'g-', 'm-']
    for i, (pred, name) in enumerate(zip(y_pred_orig_list, model_names)):
        plt.plot(pred, colors[i], label=f'{name} Predicted', alpha=0.7)

    plt.title('GHI Predictions Over Time')
    plt.xlabel('Time Step')
    plt.ylabel('GHI (W/m²)')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

    # Log the plot to wandb
    if USE_WANDB:
        wandb.init(
            project=WANDB_PROJECT,
            entity=WANDB_USERNAME,
            name="Time-Series-Predictions",
            config={"num_samples": num_samples}
        )

        # Log the matplotlib figure
        wandb.log({"time_series_predictions": wandb.Image(fig)})

        # Create an interactive line chart
        time_steps = list(range(num_samples))
        data = [[step] + [float(y_true_orig[step][0])] + [float(pred[step][0]) for pred in y_pred_orig_list]
                for step in range(num_samples)]

        columns = ["Time Step", "Actual"] + model_names
        table = wandb.Table(columns=columns, data=data)

        wandb.log({"predictions_table": table})

        # Log prediction error over time
        error_data = []
        for step in range(num_samples):
            actual = float(y_true_orig[step][0])
            errors = [abs(float(pred[step][0]) - actual) for pred in y_pred_orig_list]
            error_data.append([step, actual] + errors)

        error_columns = ["Time Step", "Actual"] + [f"{name}_error" for name in model_names]
        error_table = wandb.Table(columns=error_columns, data=error_data)

        wandb.log({"prediction_errors": error_table})

        wandb.finish()

    return fig

# Plot time series predictions
plot_predictions_over_time(
    models=[lstm_model, cnn_lstm_model, mlp_model],
    model_names=['LSTM', 'CNN-LSTM', 'MLP', 'PINN-MLP'],
    data_loader=test_loader,
    target_scaler=scalers[f'{TARGET_VARIABLE}_scaler'],
    num_samples=200,
    start_idx=100
)


## 7. Conclusion

This notebook implemented four deep learning models for GHI forecasting:

1. **LSTM Model**: Captures long-term temporal dependencies in the enhanced meteorological dataset.
2. **CNN-LSTM Model**: Combines local pattern extraction via CNNs with temporal modeling via LSTMs.
3. **MLP Model**: Processes flattened time series with many more features than the original model.
4. **PINN-MLP Model**: Enhances MLP with physics-informed constraints to enforce zero GHI at night.

### Key Findings

- The models effectively handle the expanded feature set from the new dataset.
- Physics-informed constraints improve model performance, especially during nighttime.
- Performance varies across models, with trade-offs in complexity and accuracy.

### Future Improvements

1. Incorporate more domain-specific features:
   - Cloud types and coverage
   - Aerosol optical depth
   - Solar zenith angle

2. Experiment with additional physical constraints:
   - Maximum GHI constraints based on clear sky models
   - More sophisticated atmospheric transmission models

3. Explore additional model architectures:
   - Transformer-based models for capturing long-range dependencies
   - Graph neural networks for spatial correlations
   - Ensemble methods combining multiple model types

In [ ]:
# Save trained models
def save_models_with_metadata(models, model_names, metrics_list, log_to_wandb=True):
    """
    Save trained models with performance metadata

    Args:
        models: List of PyTorch models
        model_names: List of model names
        metrics_list: List of metrics dictionaries for each model
        log_to_wandb: Whether to log models as wandb artifacts
    """
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    save_dir = f"models_{timestamp}"
    os.makedirs(save_dir, exist_ok=True)

    # Save each model with its metadata
    for model, name, metrics in zip(models, model_names, metrics_list):
        # Create unique model filename
        model_filename = f"{name.lower().replace('-', '_')}_model.pt"
        model_path = os.path.join(save_dir, model_filename)

        # Create metadata for the model
        metadata = {
            "model_name": name,
            "saved_date": timestamp,
            "performance": {
                "rmse": float(metrics['rmse']),
                "mae": float(metrics['mae']),
                "r2": float(metrics['r2']),
                "day_rmse": float(metrics['day_rmse']),
                "day_mae": float(metrics['day_mae']),
                "day_r2": float(metrics['day_r2'])
            },
            "training_config": {
                "lookback": LOOKBACK,
                "features": SELECTED_FEATURES,
                "device": str(device)
            }
        }

        # Save the model
        torch.save({
            'model_state_dict': model.state_dict(),
            'metadata': metadata
        }, model_path)

        # Create metadata JSON file
        metadata_path = os.path.join(save_dir, f"{name.lower().replace('-', '_')}_metadata.json")
        with open(metadata_path, 'w') as f:
            import json
            json.dump(metadata, f, indent=2)

        print(f"Model {name} saved to {model_path}")

        # Log as wandb artifact if enabled
        if USE_WANDB and log_to_wandb:
            wandb.init(
                project=WANDB_PROJECT,
                entity=WANDB_USERNAME,
                name=f"{name}-Model-Save",
                config=metadata
            )

            # Create artifact
            artifact = wandb.Artifact(
                name=f"{name.lower().replace('-', '_')}_model",
                type="model",
                description=f"Trained {name} model for GHI forecasting"
            )

            # Add model file to artifact
            artifact.add_file(model_path)

            # Add metadata file to artifact
            artifact.add_file(metadata_path)

            # Log artifact
            wandb.log_artifact(artifact)

            # Finish wandb run
            wandb.finish()

    print(f"All models saved successfully to {save_dir}")
    return save_dir

# Save all models with their performance metrics
models = [lstm_model, cnn_lstm_model, mlp_model]
model_names = ['LSTM', 'CNN-LSTM', 'MLP']
metrics_list = [lstm_test_metrics, cnn_lstm_test_metrics, mlp_test_metrics]

saved_models_dir = save_models_with_metadata(models, model_names, metrics_list)
print(f"Models saved to {saved_models_dir}")


### Ending Weights & Biases Session

In [ ]:
# Ensure all wandb runs are properly closed
if 'wandb' in globals() and wandb.run is not None:
    wandb.finish()
